In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import keras

import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import matplotlib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df=pd.read_csv("/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv")

In [ ]:
df.describe()

In [ ]:
df = df[df['RainTomorrow'].notna()]

In [ ]:
df.head()

In [ ]:
df['month'] = pd.DatetimeIndex(df['Date']).month
df['day'] = pd.DatetimeIndex(df['Date']).day_name()
df['date'] = pd.DatetimeIndex(df['Date']).day

In [ ]:
train_df = df.select_dtypes(exclude=['int','float'])

In [ ]:
train_df = train_df.fillna('missing')

In [ ]:
train_df = train_df.drop('Date', axis=1)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder()
for i in train_df:
  train_df[i] = ord_enc.fit_transform(train_df[[i]])

In [ ]:
for i in train_df:
  df[i] = train_df[i]

In [ ]:
df = df.fillna(-5)

In [ ]:
df = df.drop('Date', axis = 1)

In [ ]:
prediction_set = df['RainTomorrow']
df.drop('RainTomorrow',axis = 1, inplace = True)
df.drop('RainToday',axis = 1, inplace = True)
training_set = df

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(training_set , prediction_set, test_size=0.25, random_state=11)

In [ ]:
print(x_train.shape)

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras import layers, Input, Model
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

seed = 7
np.random.seed(seed)
model = Sequential()
model.add(Dense(2048, input_dim=23, activation='sigmoid'))
model.add(Dense(2048, activation='sigmoid'))
model.add(Dense(1024, activation='sigmoid'))
model.add(Dense(512, activation='sigmoid'))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(optimizer='adam',
                 loss='binary_crossentropy', metrics=['acc','mse'])

In [ ]:
df.head()

In [ ]:
model.fit(x_train, y_train, epochs=30, validation_data=(x_test, y_test), batch_size=3000, verbose=1)

In [ ]:
y = model.predict(x_test)

In [ ]:
final = pd.DataFrame(y_test)
final['prdicted'] = y
final['actual'] = final['RainTomorrow']
final = final.drop('RainTomorrow', axis = 1)

In [ ]:
final.head(30)

In [ ]:
final.to_csv('result.csv', index=False)